In [ ]:
import pandas as pd
electoral_result = pd.read_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/1976-2020-president.csv')

electoral_result = electoral_result.drop(columns=['state_po', 'state_fips', 'state_cen', 'state_ic', 'writein', 'notes', 'version'])

electoral_result = electoral_result[electoral_result['year'] >= 2008]

electoral_result.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/2008-2020-president.csv')
print(electoral_result.head())


In [5]:
winning_percentage = round((electoral_result['candidatevotes'] / electoral_result['totalvotes']) * 100, 2)
electoral_result['winning_percentage'] = winning_percentage


In [ ]:
# Drop only the columns that exist in the DataFrame
columns_to_drop = ['candidatevotes', 'totalvotes', 'party_simplified']
existing_columns_to_drop = [col for col in columns_to_drop if col in electoral_result.columns]
electoral_result = electoral_result.drop(columns=existing_columns_to_drop)

filtered_electoral_result = electoral_result[electoral_result['party_detailed'].isin(['REPUBLICAN', 'DEMOCRAT'])]


df = pd.DataFrame(filtered_electoral_result)

# Reshape the DataFrame
merged = df.groupby(['year', 'state', 'office']).apply(lambda group: pd.Series({
    'Winning Candidate': group.loc[group['winning_percentage'].idxmax(), 'candidate'],
    'Winning Party': group.loc[group['winning_percentage'].idxmax(), 'party_detailed'],
    'Winning Percentage': group['winning_percentage'].max(),
    'Losing Candidate': group.loc[group['winning_percentage'].idxmin(), 'candidate'],
    'Losing Party': group.loc[group['winning_percentage'].idxmin(), 'party_detailed'],
    'Losing Percentage': group['winning_percentage'].min()
})).reset_index()


# Rename several rows to fit the format of the other datasets
merged = merged.rename(columns={'state': 'State', 'office': 'Office', 'year': 'Year'})
# Display the merged DataFrame
print(merged)
merged.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/processed_electoral_data.csv', index=False)